In [1]:
import pandas as pd
import numpy as np
import os
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials
from hyperopt import space_eval
import hyperopt.pyll.stochastic
#import pickle
import time
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [6]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
tmp_app_train = pd.read_csv('home_credit_train.csv')
tmp_app_train = reduce_mem_usage(tmp_app_train)
tmp_app_test = pd.read_csv('home_credit_test.csv')
tmp_app_test = reduce_mem_usage(tmp_app_test)



Memory usage of dataframe is 1389.97 MB
Memory usage after optimization is: 427.19 MB
Decreased by 69.3%
Memory usage of dataframe is 220.16 MB
Memory usage after optimization is: 67.08 MB
Decreased by 69.5%


In [7]:
app_train = pd.read_csv('app_train_new.csv')
app_test = pd.read_csv('app_test_new.csv')

app_train = reduce_mem_usage(app_train)
app_test = reduce_mem_usage(app_test)

Memory usage of dataframe is 18.75 MB
Memory usage after optimization is: 5.86 MB
Decreased by 68.7%
Memory usage of dataframe is 2.98 MB
Memory usage after optimization is: 0.93 MB
Decreased by 68.7%


In [ ]:
tmp_app_train.shape

In [5]:
tmp_app_test.shape

(48744, 592)

###### Joining with application new fields

In [8]:
tmp_app_train = pd.merge(tmp_app_train,app_train,on=['SK_ID_CURR'],how='inner')
tmp_app_test = pd.merge(tmp_app_test,app_test,on=['SK_ID_CURR'],how='inner')

In [9]:
prev = pd.read_csv('previous_default_rates.csv')
prev = reduce_mem_usage(prev)

Memory usage of dataframe is 12.93 MB
Memory usage after optimization is: 3.88 MB
Decreased by 70.0%


In [10]:
tmp_app_train = pd.merge(tmp_app_train,prev,on=['SK_ID_CURR'],how='left')
tmp_app_test = pd.merge(tmp_app_test,prev,on=['SK_ID_CURR'],how='left')

In [11]:
tmp_app_train = tmp_app_train.fillna(-1)
tmp_app_test = tmp_app_test.fillna(-1)

In [12]:
tmp_app_train.shape

(307228, 604)

In [13]:
tmp_app_test.shape

(48744, 603)

##### Removing features with unique value =1

In [14]:
def remove_col(df):
    remove_col_list =[]
    for col in df.columns:
        if (len(df[col].unique()))==1:
            remove_col_list.append(col)
    return remove_col_list

In [15]:
# VAR_SK_DPD , VAR_SK_DPD_DEF

In [16]:
remove_col_list = remove_col(tmp_app_train)
remove_col_list

['FLAG_MOBIL']

In [17]:
tmp_app_train = tmp_app_train.drop(columns=remove_col_list)
tmp_app_test = tmp_app_test.drop(columns=remove_col_list)

##### Adding features

In [18]:
tmp_app_train['OBS_TOTAL'] = tmp_app_train['OBS_30_CNT_SOCIAL_CIRCLE'] + tmp_app_train['OBS_60_CNT_SOCIAL_CIRCLE']
tmp_app_train['DEF_TOTAL'] = tmp_app_train['DEF_30_CNT_SOCIAL_CIRCLE'] + tmp_app_train['DEF_60_CNT_SOCIAL_CIRCLE']

tmp_app_test['OBS_TOTAL'] = tmp_app_test['OBS_30_CNT_SOCIAL_CIRCLE'] + tmp_app_test['OBS_60_CNT_SOCIAL_CIRCLE']
tmp_app_test['DEF_TOTAL'] = tmp_app_test['DEF_30_CNT_SOCIAL_CIRCLE'] + tmp_app_test['DEF_60_CNT_SOCIAL_CIRCLE']


In [19]:
tmp_app_train['OBS_DIFF'] = tmp_app_train['OBS_30_CNT_SOCIAL_CIRCLE'] - tmp_app_train['OBS_60_CNT_SOCIAL_CIRCLE']
tmp_app_test['OBS_DIFF'] = tmp_app_test['OBS_30_CNT_SOCIAL_CIRCLE'] - tmp_app_test['OBS_60_CNT_SOCIAL_CIRCLE']

tmp_app_train['DEF_DIFF'] = tmp_app_train['DEF_30_CNT_SOCIAL_CIRCLE'] - tmp_app_train['DEF_60_CNT_SOCIAL_CIRCLE']
tmp_app_test['DEF_DIFF'] = tmp_app_test['DEF_30_CNT_SOCIAL_CIRCLE'] - tmp_app_test['DEF_60_CNT_SOCIAL_CIRCLE']



In [20]:
tmp_app_train['RATIO_SOCIAL'] = tmp_app_train['DEF_TOTAL'] / (1+tmp_app_train['OBS_TOTAL'])
tmp_app_train['RATIO_SOCIAL_DIFF'] = tmp_app_train['DEF_DIFF'] / (1+tmp_app_train['OBS_DIFF'])

tmp_app_test['RATIO_SOCIAL'] = tmp_app_test['DEF_TOTAL'] / (1+tmp_app_test['OBS_TOTAL'])
tmp_app_test['RATIO_SOCIAL_DIFF'] = tmp_app_test['DEF_DIFF'] / (1+tmp_app_test['OBS_DIFF'])

In [21]:
tmp_app_train.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_11,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DOCUMENT_IND_MEAN,DOCUMENT_IND_STD,DOCUMENT_IND_KURT,DOCUMENT_IND_SUM,FLAG_IND_SUM,FLAG_IND_KURT,FLAG_IND_STD,FLAG_IND_MEAN,EXT_SOURCE_MISSING,ORG_CORRECT,ORG_DEFAULT,DEFAULT_RATE_ORG_TYPE,NAME_TYPE_CORRECT,NAME_TYPE_DEFAULT,DEFAULT_RATE_NAME_TYPE,OCCUPATION_TYPE_CORRECT,OCCUPATION_TYPE_DEFAULT,DEFAULT_RATE_OCC_TYPE,INCOME_TYPE_CORRECT,INCOME_TYPE_DEFAULT,DEFAULT_RATE_INCOME_TYPE,EDUCATION_TYPE_CORRECT,EDUCATION_TYPE_DEFAULT,DEFAULT_RATE_EDUCATION_TYPE,FAMILY_STATUS_CORRECT,FAMILY_STATUS_DEFAULT,DEFAULT_RATE_FAMILY_STATUS,HOUSING_TYPE_CORRECT,HOUSING_TYPE_DEFAULT,DEFAULT_RATE_HOUSING_TYPE,TOTALAREA_NAN,EXT_SOURCE_3_NAN,EMERGENCYSTATE_NAN,AMT_REQ_CREDIT_BUREAU_NAN,DEF_SOCIAL_CIRCLE_NAN,COMMONAREA_NAN,NONLIVINGAPARTMENTS_NAN,LIVINGAPARTMENTS_NAN,EXT_SOURCE_1_NAN,EXT_SOURCE_2_NAN,NONLIVINGAREA_NAN,YEARS_BEGINEXPLUATATION_NAN,AMT_ANNUITY_NAN,APARTMENT_FLAG,DAYS_EMPLOYED_ANOM,NAME_CONTRACT_TYPE_Cash loans_x,NAME_CONTRACT_TYPE_Revolving loans_x,TARGET,BUREAU_MONTHS_x,LATEST_STATUS_0,LATEST_STATUS_1,LATEST_STATUS_2,LATEST_STATUS_3,LATEST_STATUS_4,LATEST_STATUS_5,LATEST_STATUS_C,LATEST_STATUS_X,DAYS_CREDIT_UPDATE,TOTAL_SUM_AMT_CREDIT_SUM,TOTAL_SUM_AMT_CREDIT_SUM_DEBT,BUREAU_MAX_AMT_CREDIT_MAX_OVERDUE,BUREAU_MAX_AMT_CREDIT_SUM,BUREAU_MAX_AMT_CREDIT_SUM_DEBT,BUREAU_MAX_AMT_CREDIT_SUM_LIMIT,BUREAU_MAX_AMT_CREDIT_SUM_OVERDUE,BADDEBT_SUM_AMT_CREDIT_SUM,BADDEBT_SUM_AMT_CREDIT_SUM_DEBT,CLOSED_SUM_AMT_CREDIT_SUM,CLOSED_SUM_AMT_CREDIT_SUM_DEBT,ACTIVE_SUM_AMT_CREDIT_SUM,ACTIVE_SUM_AMT_CREDIT_SUM_DEBT,BUREAU_AMT_ANNUITY_SUM,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,BUREAU_SUM_CNT_CREDIT_PROLONG,BUREAU_CNT_CREDIT_TYPE_Another type of loan,BUREAU_CNT_CREDIT_TYPE_Car loan,BUREAU_CNT_CREDIT_TYPE_Cash loan (non-earmarked),BUREAU_CNT_CREDIT_TYPE_Consumer credit,BUREAU_CNT_CREDIT_TYPE_Credit card,BUREAU_CNT_CREDIT_TYPE_Interbank credit,BUREAU_CNT_CREDIT_TYPE_Loan for business development,BUREAU_CNT_CREDIT_TYPE_Loan for purchase of shares (margin lending),BUREAU_CNT_CREDIT_TYPE_Loan for the purchase of equipment,BUREAU_CNT_CREDIT_TYPE_Loan for working capital replenishment,BUREAU_CNT_CREDIT_TYPE_Microloan,BUREAU_CNT_CREDIT_TYPE_Mobile operator loan,BUREAU_CNT_CREDIT_TYPE_Mortgage,BUREAU_CNT_CREDIT_TYPE_Real estate loan,BUREAU_CNT_CREDIT_TYPE_Unknown type of loan,BUREAU_CNT_CREDIT_ACTIVE_Active,BUREAU_CNT_CREDIT_ACTIVE_Bad debt,BUREAU_CNT_CREDIT_ACTIVE_Closed,BUREAU_CNT_CREDIT_ACTIVE_Sold,BUREAU_TOTAL_COUNT,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,BUREAU_MONTHS_y,BUREAU_FLAG,LATEST_LATEST_MONTHS_BALANCE_x,LATEST_LATEST_AMT_BALANCE_x,LATEST_LATEST_AMT_CREDIT_LIMIT_ACTUAL_x,LATEST_LATEST_AMT_DRAWINGS_ATM_CURRENT_x,LATEST_LATEST_AMT_DRAWINGS_OTHER_CURRENT_x,LATEST_LATEST_AMT_DRAWINGS_POS_CURRENT_x,LATEST_LATEST_AMT_PAYMENT_CURRENT_x,LATEST_LATEST_CNT_DRAWINGS_ATM_CURRENT_x,LATEST_LATEST_CNT_DRAWINGS_CURRENT_x,LATEST_LATEST_CNT_DRAWINGS_OTHER_CURRENT_x,LATEST_LATEST_CNT_INSTALMENT_MATURE_CUM_x,LATEST_LATEST_SK_DPD_x,LAT

In [59]:
tmp_app_test.shape

(48744, 608)

In [60]:
tmp_app_train.shape

(307228, 609)

In [24]:
remove_col_list = remove_col(tmp_app_test)
remove_col_list

[]

In [61]:
train =  tmp_app_train.drop(columns=no_imp_feat)
test = tmp_app_test.drop(columns=no_imp_feat)

In [62]:
#tmp_app_train = tmp_app_train.sample(n=100000)
X = train.drop(columns = ['SK_ID_CURR','TARGET']).values
y = train['TARGET'].values

In [94]:
# train = tmp_app_train.drop(columns=list(f_imp[f_imp==0].index))
test = tmp_app_test.drop(columns=list(f_imp[f_imp==0].index))

In [67]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)

In [97]:
test = pd.read_csv('test.csv')

In [98]:
test.shape

(48744, 503)

##### We need to divide training data to train, validation and test in the ratio 0.7,0.2,0.1

In [82]:
from sklearn.model_selection import train_test_split
def split_data(X,y,size):
    # test_size ideal 0.2 ; random_state can be any value
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=size,random_state=42)
    return X_train,X_test,y_train,y_test
############################################

In [83]:
X_train,X_val,y_train,y_val = split_data(X,y,0.3)
X_val,X_test,y_val,y_test = split_data(X_val,y_val,.33)

##### We use this to scale the data

In [66]:
from sklearn.preprocessing import StandardScaler 
sc_X = StandardScaler()


In [69]:
X = sc_X.fit_transform(X)

In [81]:
X.shape

(307228, 502)

In [84]:
X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)
X_val = sc_X.transform(X_val)

In [29]:
# X_train = pd.DataFrame(sc_X.fit_transform(X_train),columns=X_train.columns)

In [30]:
# X_val = pd.DataFrame(sc_X.fit_transform(X_val),columns=X_val.columns)
# X_test = pd.DataFrame(sc_X.fit_transform(X_test),columns=X_test.columns)

In [31]:
# print('training shape: '+ str(X_train.shape))
# print('Cross val shape: '+ str(X_val.shape))
# print('Test shape: '+ str(X_test.shape))

training shape: (276505, 607)
Cross val shape: (20584, 607)
Test shape: (10139, 607)


In [ ]:
np.unique(y,return_counts = True)

In [68]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

### Parameter tuning

In [74]:
def objective(params):
    params = {
      'bagging_fraction': '{:.5f}'.format(params['bagging_fraction']),
         'learning_rate': '{:.5f}'.format(params['learning_rate']),
         'colsample_bytree': '{:.4f}'.format(params['colsample_bytree']),        
        'max_bin': int(params['max_bin']),
        'num_leaves': int(params['num_leaves']),
        'n_estimators': int(params['n_estimators']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'feature_fraction': '{:.4f}'.format(params['feature_fraction']), 
        'min_gain_to_split': '{:.4f}'.format(params['min_gain_to_split']),
        'reg_alpha': '{:.4f}'.format(params['reg_alpha']),
        'reg_lambda': '{:.4f}'.format(params['reg_lambda']),
        'max_depth': int(params['max_depth']),
        'seed': 0,
        'min_child_weight' : int(params['min_child_weight']),
        'scale_pos_weight' : 0.087
        }
    clf = LGBMClassifier(
        
        **params
    )
    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    results = cross_val_score(clf, X, y, cv=kfold,scoring='roc_auc')
  
    #clf.fit(X_train, y_train)    
 
    #y_pred_prob_val = clf.predict_proba(X_val)[:,1]
    #y_pred_prob_train = clf.predict_proba(X_train)[:,1]
    #y_pred_prob_test = clf.predict_proba(X_test)[:,1]
    #auc_score_val = roc_auc_score(y_val, y_pred_prob_val)
    #auc_score_train = roc_auc_score(y_train, y_pred_prob_train)
    #auc_score_test = roc_auc_score(y_test, y_pred_prob_test)
    print(results,'mean auc score: ',results.mean())
    print(params)
    #print("val_score {:.5f} \t train_score {:.5f} \t test_score: {:.5f} \nparams {}".format(auc_score_val,auc_score_train,auc_score_test, params))
    return (1-results.mean())
   #return (1-auc_score_val)

In [75]:
space = {
      'bagging_fraction' : hp.uniform('bagging_fraction', .3, 0.9),
       'learning_rate' : hp.uniform('learning_rate', .01, .1),
     'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 0.9),
    'max_bin': hp.quniform('max_bin', 120, 420, 5),
    'num_leaves': hp.quniform('num_leaves', 20, 140, 2),
    'n_estimators': hp.quniform('n_estimators', 1200, 2100, 20),
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 10, 280, 10),
        'feature_fraction': hp.uniform('feature_fraction', 0.2, 0.9),
        'min_gain_to_split': hp.uniform('min_gain_to_split', 0.1, 0.9),
           'reg_alpha': hp.uniform('reg_alpha', .01, .1),
        'reg_lambda': hp.uniform('reg_lambda', .01, .1),
        'max_depth': hp.quniform('max_depth', -1, 30, 1),
        'seed': 0,
        'min_child_weight' : hp.quniform('min_child_weight', 50, 150, 10)
}

In [76]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30)

[0.77952877 0.78784819 0.78987475] mean auc score:  0.7857505726128915
{'bagging_fraction': '0.31928', 'learning_rate': '0.07383', 'colsample_bytree': '0.5603', 'max_bin': 280, 'num_leaves': 108, 'n_estimators': 1620, 'min_data_in_leaf': 50, 'feature_fraction': '0.7409', 'min_gain_to_split': '0.4740', 'reg_alpha': '0.0755', 'reg_lambda': '0.0435', 'max_depth': 7, 'seed': 0, 'min_child_weight': 80, 'scale_pos_weight': 0.087}
[0.77666496 0.78521207 0.7878154 ] mean auc score:  0.7832308092458188
{'bagging_fraction': '0.59659', 'learning_rate': '0.08438', 'colsample_bytree': '0.4655', 'max_bin': 240, 'num_leaves': 28, 'n_estimators': 1480, 'min_data_in_leaf': 160, 'feature_fraction': '0.8498', 'min_gain_to_split': '0.5884', 'reg_alpha': '0.0329', 'reg_lambda': '0.0467', 'max_depth': 22, 'seed': 0, 'min_child_weight': 130, 'scale_pos_weight': 0.087}
[0.77823097 0.78665701 0.78924301] mean auc score:  0.7847103268085408
{'bagging_fraction': '0.46840', 'learning_rate': '0.08604', 'colsample_

In [ ]:
 best = {'bagging_fraction': '0.68136', 'learning_rate': '0.02325',
         'colsample_bytree': '0.7283', 'max_bin': 180, 'num_leaves': 48, 
         'n_estimators': 1300, 'min_data_in_leaf': 230, 'feature_fraction': '0.2674',
         'min_gain_to_split': '0.5626', 'reg_alpha': '0.0791', 'reg_lambda': '0.0606',
         'max_depth': 16, 'seed': 0, 'min_child_weight': 120}

In [ ]:
#val_score 0.79288 	 train_score 0.88460 	 test_score: 0.78729
best = {'bagging_fraction': '0.67174', 'learning_rate': '0.02449',
        'colsample_bytree': '0.6401', 'max_bin': 315, 'num_leaves': 44, 
        'n_estimators': 1500, 'min_data_in_leaf': 130, 'feature_fraction': '0.2692',
        'min_gain_to_split': '0.8799', 'reg_alpha': '0.0351', 'reg_lambda': '0.0508',
        'max_depth': 14, 'seed': 0, 'subsample_for_bin':800000,
        'min_child_weight': 130, 'subsample_frequency': '0.83434'}

In [ ]:
# best = {'bagging_fraction': '0.50064', 'learning_rate': '0.02807', 
#         'colsample_bytree': '0.4822', 'max_bin': 390, 'num_leaves': 26, 
#         'n_estimators': 1800, 'min_data_in_leaf': 120, 'feature_fraction': '0.3009',
#         'min_gain_to_split': '0.3432', 'reg_alpha': '0.0547', 
#         'reg_lambda': '0.0410', 'max_depth': 22, 'seed': 0, 'min_child_weight': 90 }
#         #'subsample_frequency': '0.77138'}

In [ ]:
# best = {'bagging_fraction': '0.78328', 'learning_rate': '0.03708', 
#         'colsample_bytree': '0.4451', 'max_bin': 385, 'num_leaves': 28,
#         'n_estimators': 1500, 'min_data_in_leaf': 170, 'feature_fraction': '0.2330',
#         'min_gain_to_split': '0.1414', 'reg_alpha': '0.0415', 'reg_lambda': '0.0549',
#         'max_depth': 15, 'seed': 0, 'min_child_weight': 100, 
#         'subsample_frequency': '0.77583'}

In [77]:
best_params = space_eval(space, best)

In [78]:
best_params

{'bagging_fraction': 0.40650893650961334,
 'colsample_bytree': 0.8966358403650636,
 'feature_fraction': 0.21407021825056116,
 'learning_rate': 0.024217122627912854,
 'max_bin': 150.0,
 'max_depth': 19.0,
 'min_child_weight': 50.0,
 'min_data_in_leaf': 210.0,
 'min_gain_to_split': 0.12341280434109966,
 'n_estimators': 1740.0,
 'num_leaves': 98.0,
 'reg_alpha': 0.045042345994512505,
 'reg_lambda': 0.06385424987056496,
 'seed': 0}

In [79]:
best = {'bagging_fraction': 0.40650893650961334,
 'colsample_bytree': 0.8966358403650636,
 'feature_fraction': 0.21407021825056116,
 'learning_rate': 0.024217122627912854,
 'max_bin': 150,
 'max_depth': 19,
 'min_child_weight': 50,
 'min_data_in_leaf': 210,
 'min_gain_to_split': 0.12341280434109966,
 'n_estimators': 1740,
 'num_leaves': 98,
 'reg_alpha': 0.045042345994512505,
 'reg_lambda': 0.06385424987056496,
 'seed': 0}

In [91]:
classifier = LGBMClassifier(
    **best
                           )
classifier.fit(X, y)

LGBMClassifier(bagging_fraction=0.40650893650961334, boosting_type='gbdt',
        class_weight=None, colsample_bytree=0.8966358403650636,
        feature_fraction=0.21407021825056116,
        learning_rate=0.024217122627912854, max_bin=150, max_depth=19,
        min_child_samples=20, min_child_weight=50, min_data_in_leaf=210,
        min_gain_to_split=0.12341280434109966, min_split_gain=0.0,
        n_estimators=1740, n_jobs=-1, num_leaves=98, objective=None,
        random_state=None, reg_alpha=0.045042345994512505,
        reg_lambda=0.06385424987056496, seed=0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [90]:
y

array([1, 0, 0, ..., 0, 1, 0], dtype=int8)

#### FEATURE IMPORTANCE

In [44]:
# f_imp_2 = pd.Series(classifier.booster_.feature_importance(importance_type='gain'), index = X_train.columns)

In [45]:
# f_imp_2.sort_values(ascending=False).reset_index().to_csv('feature_importance_2.csv',index=False)

In [46]:
# f_imp_2.sort_values(ascending=False)

EXT_SOURCE_mean                                                        48226.532679
EXT_SOURCE sum                                                         17731.448555
NEW_CREDIT_TO_ANNUITY_RATIO                                            17715.334166
EXT_SOURCE_2  EXT_SOURCE_3                                             17029.879327
EXT_SOURCE_1  EXT_SOURCE_3                                             16460.827658
EXT_SOURCE / mean                                                      14965.259868
EXT_SOURCE prod                                                        11818.507584
EXT_SOURCE_2 / DAYS_BIRTH_x                                             9441.717042
EXT_SOURCE_1  EXT_SOURCE_2                                              9322.067615
EXT_SOURCE_3 / DAYS_BIRTH_x                                             9103.344234
EXT_SOURCE_1 / EXT_SOURCE_2                                             8828.068591
DAYS_BIRTH                                                              7561

In [48]:
# len(list(f_imp[f_imp==0].index))

126

In [ ]:
#list2=pd.read_csv('feature_importance_2.csv')

In [ ]:
# list(list2[list2['0']==0]['index'])

In [49]:
# len(set(list(f_imp_2[f_imp_2==0].index)).intersection(set(list(f_imp[f_imp==0].index))))

105

In [52]:
# no_imp_feat = list(set(list(f_imp_2[f_imp_2==0].index)).intersection(set(list(f_imp[f_imp==0].index))))

In [86]:
y_pred_prob_test = classifier.predict_proba(X_test)[:,1]
y_pred_prob_train = classifier.predict_proba(X_train)[:,1]
y_pred_prob_val = classifier.predict_proba(X_val)[:,1]
print("cross validation ROC score: "+str(roc_auc_score(y_val, y_pred_prob_val)))
print("train validation ROC score: "+str(roc_auc_score(y_train, y_pred_prob_train)))
print("test validation ROC score: "+str(roc_auc_score(y_test, y_pred_prob_test)))

cross validation ROC score: 0.7920794687487851
train validation ROC score: 0.9800075384508335
test validation ROC score: 0.7863595131478331


In [ ]:
cross validation ROC score: 0.9137931034482758
train validation ROC score: 0.8705278275836836

cross validation ROC score: 0.9310344827586207
train validation ROC score: 0.8705024149628342

cross validation ROC score: 0.9482758620689655
train validation ROC score: 0.8704024642319563

In [ ]:
cross validation ROC score: 0.794314762557148
train validation ROC score: 0.8779015198010918

    
cross validation ROC score: 0.7945101703751719
train validation ROC score: 0.8934470093237312

cross validation ROC score: 0.7945233962936337
train validation ROC score: 0.8931255026150717
    
cross validation ROC score: 0.7940571775008592
train validation ROC score: 0.8870916546568616

In [ ]:
y_val

In [ ]:
y_pred_prob_val

In [ ]:
cross validation ROC score: 0.7964193387212095
train validation ROC score: 0.9228320299500886
test validation ROC score: 0.7903973805554734  - 0.796

cross validation ROC score: 0.7962487655738522
train validation ROC score: 0.9099766559268216
test validation ROC score: 0.7902121371458123   - 

In [ ]:
np.unique(y_train,return_counts=True)

In [ ]:
cross validation ROC score: 0.7965079835029962
train validation ROC score: 0.9047996835447147
test validation ROC score: 0.7896592733525682

cross validation ROC score: 0.7959102995391298
train validation ROC score: 0.8701826256905496
test validation ROC score: 0.790035373608547

cross validation ROC score: 0.7958931388679629
train validation ROC score: 0.8669098756140712
test validation ROC score: 0.7893263039968563
    
cross validation ROC score: 0.7964702776754957
train validation ROC score: 0.9065846432128992
test validation ROC score: 0.7895785055538685
    
cross validation ROC score: 0.7958607164810749
train validation ROC score: 0.8948006588412571
test validation ROC score: 0.7892271133767896

cross validation ROC score: 0.7961526581073808
train validation ROC score: 0.9182231590583936
test validation ROC score: 0.7891496300357079



In [100]:
test.shape

(48744, 503)

##### Predicting the test file

In [101]:
# #Predicting TARGET in test data set
X = test.drop(columns = ['SK_ID_CURR'])
X = sc_X.transform(X)
target = classifier.predict(X)
target_prob = classifier.predict_proba(X)[:,1]

C:\Users\Leo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


##### Saving the submission file

In [102]:
test_sk_id = tmp_app_test['SK_ID_CURR']
submit_df = pd.DataFrame(data = list(zip(test_sk_id,target_prob)),columns = ['SK_ID_CURR','TARGET'])
submit_df.to_csv('submission_lgbm.csv',index = False)